# Logging

- [Python JSON Logger](https://github.com/nhairs/python-json-logger)

In [1]:
import logging

logging.debug('This is a debug message')
logging.info('This is a info message')
logging.warning('This is a warning message')
logging.error('This is a error message')
logging.critical('This is a critical message')

ERROR:root:This is a error message
CRITICAL:root:This is a critical message


In [5]:
import logging
#logging.basicConfig(level=logging.DEBUG,format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',datefmt='%Y-%m-%d %H:%M:%S')
logging.basicConfig(level=logging.DEBUG)

logging.debug('This is a debug message')
logging.info('This is a info message')
logging.warning('This is a warning message')
logging.error('This is a error message')
logging.critical('This is a critical message')

ERROR:root:This is a error message
CRITICAL:root:This is a critical message


## Logger in SQLMonitor Alert Engine with RotatingFileHandler

In [1]:
import logging
import os
import sys
from logging.handlers import RotatingFileHandler

def get_script_logger(script_name:str,log_file:str=None):
    # create logger
    logger = logging.getLogger(script_name)
    logger.setLevel(logging.DEBUG)

    # create console handler and set level to debug
    sh = logging.StreamHandler(sys.stdout)
    sh.setLevel(logging.DEBUG)

    # create formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

    # add formatter to sh
    sh.setFormatter(formatter)

    if log_file:
        #log_file_path = f"SQLMonitor-AlertEngine-Logs.log"
        max_log_size = 5 * 1024 * 1024  # 5 MB
        backup_count = 4  # Number of backup logs to keep

        # Ensure to create Logs directory is not exists
        log_dir = os.path.dirname(log_file)
        if not os.path.exists(log_dir):
            os.makedirs(log_dir)

        # Create a RotatingFileHandler
        fh = RotatingFileHandler(
            log_file, maxBytes=max_log_size, backupCount=backup_count
        )

        #fh = logging.FileHandler(log_file)
        fh.setLevel(logging.DEBUG)
        fh.setFormatter(formatter)
        logger.addHandler(fh)
        #print(f"\n[get_script_logger()] => Logging to file '{log_file}'.\n")
    else:
        # add sh to logger
        #print(f"[get_script_logger()] => Using console logger..")
        logger.addHandler(sh)

    return logger

In [6]:
steamLogger = get_script_logger('Logging-module.ipynb')
steamLogger.info('This is testing of steam logger')

fileLogger = get_script_logger('Logging-module.ipynb','/tmp/logging-module.log')
fileLogger.info('This is testing of file logger')
fileLogger.info('I dont have anything else to log')

2025-05-23 08:22:21,771 - Logging-module.ipynb - INFO - This is testing of steam logger
2025-05-23 08:22:21,771 - Logging-module.ipynb - INFO - This is testing of steam logger
2025-05-23 08:22:21,771 - Logging-module.ipynb - INFO - This is testing of steam logger
2025-05-23 08:22:21,771 - Logging-module.ipynb - INFO - This is testing of steam logger
2025-05-23 08:22:21,773 - Logging-module.ipynb - INFO - This is testing of file logger
2025-05-23 08:22:21,773 - Logging-module.ipynb - INFO - This is testing of file logger
2025-05-23 08:22:21,773 - Logging-module.ipynb - INFO - This is testing of file logger
2025-05-23 08:22:21,773 - Logging-module.ipynb - INFO - This is testing of file logger
2025-05-23 08:22:21,774 - Logging-module.ipynb - INFO - I dont have anything else to log
2025-05-23 08:22:21,774 - Logging-module.ipynb - INFO - I dont have anything else to log
2025-05-23 08:22:21,774 - Logging-module.ipynb - INFO - I dont have anything else to log
2025-05-23 08:22:21,774 - Logging

# logging.conf

In [ ]:
[loggers]
keys=root,simpleExample

[handlers]
keys=consoleHandler

[formatters]
keys=simpleFormatter

[logger_root]
level=DEBUG
handlers=consoleHandler

[logger_simpleExample]
level=DEBUG
handlers=consoleHandler
qualname=simpleExample
propagate=0

[handler_consoleHandler]
class=SteamHandler
level=DEBUG
formatter=simpleFormatter
args=(sys.stdout,)

[formatter_simpleFormatter]
format=%(asctime)s - %(name)s - %(levelname)s - %(message)s

### Use logging.conf

In [ ]:
import logging
import logging.config

logging.config.fileConfig('logging.conf')
#logging.config.dictConfig('dict-logging.conf')

logger = logging.getLogger('simpleExample')
logger.debug('this is a debug message')

# dict-logging.conf

In [1]:
LOGGING_CONFIG = { 
    'version': 1,
    'disable_existing_loggers': True,
    'formatters': { 
        'standard': { 
            'format': '%(asctime)s [%(levelname)s] %(name)s: %(message)s'
        },
    },
    'handlers': { 
        'default': { 
            'level': 'INFO',
            'formatter': 'standard',
            'class': 'logging.StreamHandler',
            'stream': 'ext://sys.stdout',  # Default is stderr
        },
    },
    'loggers': { 
        '': {  # root logger
            'handlers': ['default'],
            'level': 'WARNING',
            'propagate': False
        },
        'my.packg': { 
            'handlers': ['default'],
            'level': 'INFO',
            'propagate': False
        },
        '__main__': {  # if __name__ == '__main__'
            'handlers': ['default'],
            'level': 'DEBUG',
            'propagate': False
        },
    } 
}

### Using Dict Config for logging

In [3]:
import logging
import logging.config

# Run once at startup:
logging.config.dictConfig(LOGGING_CONFIG)

# Include in each module:
log = logging.getLogger(__name__)
log.debug("Logging is configured.")

# Log Stack Trace in logging

### with known exceptions

In [7]:
import logging

try:
    a = [1,2,3]
    val = a[4]
#except IndexError as e:
except Exception as e:
    logging.error(e, exc_info=True)

2025-05-25 16:50:55,745 [ERROR] root: list index out of range
Traceback (most recent call last):
  File "/var/folders/kl/qb1k68ds67n5ptrl1yd6xk5m0000gp/T/ipykernel_51429/2940446555.py", line 5, in <module>
    val = a[4]
          ~^^^
IndexError: list index out of range


### without known exceptions

In [6]:
import logging
import traceback

try:
    a = [1,2,3]
    val = a[4]
except:
    logging.error("The error is %s", traceback.format_exc())

2025-05-25 16:50:27,655 [ERROR] root: The error is Traceback (most recent call last):
  File "/var/folders/kl/qb1k68ds67n5ptrl1yd6xk5m0000gp/T/ipykernel_51429/4070056426.py", line 6, in <module>
    val = a[4]
          ~^^^
IndexError: list index out of range



# TimedRotatingFileHandler

In [8]:
import logging
import time
from logging.handlers import TimedRotatingFileHandler

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# s, m, h, d, midnight, w0 (monday), w1, ..
handler = TimedRotatingFileHandler('timed_test.log', when='s', interval=5, backupCount=5)
logger.addHandler(handler)

for _ in range(6):
    logger.info('Hello, World!')
    time.sleep(5)

2025-05-25 16:59:23,879 [INFO] __main__: Hello, World!
2025-05-25 16:59:28,885 [INFO] __main__: Hello, World!
2025-05-25 16:59:33,890 [INFO] __main__: Hello, World!
2025-05-25 16:59:38,897 [INFO] __main__: Hello, World!
2025-05-25 16:59:43,903 [INFO] __main__: Hello, World!
2025-05-25 16:59:48,910 [INFO] __main__: Hello, World!
